In [1]:
import os, itertools

from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from classifiers import MNIST_CNN
from trainers import Trainer
from attacks import GM

os.environ["CUDA_VISIBLE_DEVICES"]="2"

fmnist = input_data.read_data_sets("FMNIST_data/")

data_train = (np.reshape(fmnist.train.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.train.labels)
data_test = (np.reshape(fmnist.test.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.test.labels)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting FMNIST_data/train-images-idx3-ubyte.gz
Extracting FMNIST_data/train-labels-idx1-ubyte.gz
Extracting FMNIST_data/t10k-images-idx3-ubyte.gz
Extracting FMNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def params_maker(param_names, param_values):
    
    product = itertools.product(*param_values)
    params = [list(zip(param_names, p)) for p in product]
    
    return params

In [3]:
use_batchnorm = False
droprate = None
dbrate = None
rmrate = None
snbeta = None
l2rate = None
gprate = None
orrate = None
lprate = None

attack = GM

attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[0.3], [0.02], [40], ['inf'], ['xent', 'cw']])

for attack_params in attack_paramss:

    logdir = 'tf_logs/exp2/'
    if use_batchnorm : logdir += 'batchnorm/'
    if droprate : logdir += 'dropout/{}/'.format(droprate)
    if dbrate : logdir += 'db/{}/'.format(dbrate)
    if rmrate : logdir += 'rm/{}/'.format(rmrate)
    if snbeta : logdir += 'sn/{}/'.format(snbeta)
    if l2rate : logdir += 'l2/{}/'.format(l2rate)
    if gprate : logdir += 'gp/{}/'.format(gprate)
    if orrate : logdir += 'or/{}/'.format(orrate)
    if lprate : logdir += 'lp/{}/'.format(lprate)

    if attack : logdir += 'adv/{}/{}/'.format(attack.__name__, '_'.join([str(v) for k, v in attack_params]))

    if logdir.split('/')[-2] == 'exp2' : logdir += 'standard/'

    print(logdir)

    tf.reset_default_graph()

    DNN = MNIST_CNN(logdir, tf.nn.relu, use_batchnorm, droprate, dbrate, rmrate, snbeta, l2rate, gprate, orrate, lprate, attack, dict(attack_params))

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    trainer = Trainer(sess, DNN, data_train)
    trainer.train(n_epochs=5, p_epochs=1)

    sess.close()
    
    # Test Accuracy
    tf.reset_default_graph()
    
    sess = tf.InteractiveSession()
    
    DNN = MNIST_CNN(logdir, tf.nn.relu)
    DNN.load(sess)
    
    print('Test Accuracy : {:.5f}'.format(DNN.evaluate(sess, data_test)))
    
    sess.close()

tf_logs/exp3/adv/GM/0.3_0.02_40_inf_xent/
Epoch : 1   | Loss : 1.08893 | Accuracy : 0.56560 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 2   | Loss : 0.83661 | Accuracy : 0.66189 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 3   | Loss : 0.75822 | Accuracy : 0.69435 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 4   | Loss : 0.71450 | Accuracy : 0.71158 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 5   | Loss : 0.67876 | Accuracy : 0.72593 | Test Accuracy : nan | Adv Accuracy : nan
INFO:tensorflow:Restoring parameters from tf_logs/exp3/adv/GM/0.3_0.02_40_inf_xent/model
Test Accuracy : 0.84370
tf_logs/exp3/adv/GM/0.3_0.02_40_inf_cw/
Epoch : 1   | Loss : 1.02658 | Accuracy : 0.55504 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 2   | Loss : 0.79704 | Accuracy : 0.65602 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 3   | Loss : 0.71014 | Accuracy : 0.69765 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 4   | Loss : 0.66816 | Accuracy : 0.71780 | Test Accu